In [1]:
from collections import Counter, defaultdict
import math, random, re, glob

In [2]:

def get_subject_data(path):
    data = []

    # regex for stripping out the leading "Subject:" and any spaces after it
    subject_regex = re.compile(r"^Subject:\s+")

    # glob.glob returns every filename that matches the wildcarded path
    for fn in glob.glob(path):
        is_spam = "ham" not in fn

        with open(fn,'r',encoding='ISO-8859-1') as file:
            for line in file:
                if line.startswith("Subject:"):
                    subject = subject_regex.sub("", line).strip()
                    data.append((subject, is_spam))
                    # 폴더에서 하나의 파일의 제목을 subject에 그 파일의 스팸 여부를 data에 저장합니다.

    return data

In [3]:
data = get_subject_data(r"data/*/*")
data[:5]

[('Re: New Sequences Window', False),
 ('[zzzzteana] RE: Alexander', False),
 ('[zzzzteana] Moscow bomber', False),
 ("[IRR] Klez: The Virus That  Won't Die", False),
 ('Re: Insert signature', False)]

In [32]:
text_data = []
result_data = []
for line in data:
    text_data.append(line[0])
    result_data.append(line[1])
    
text_data[0], result_data[0]

('Re: New Sequences Window', False)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.pipeline import Pipeline

count_ngrams = CountVectorizer(ngram_range=(1,3), analyzer="word", binary="False", stop_words="english")
X = count_ngrams.fit_transform(text_data)
X.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [60]:
import numpy as np

Y = [1 if data == True else 0 for data in result_data  ]
Y = np.array(Y)
Y

array([0, 0, 0, ..., 1, 1, 1])

In [64]:
from sklearn.cross_validation import KFold
from sklearn import linear_model, datasets
from sklearn.metrics import precision_recall_fscore_support

kf = KFold(Y.shape[0], n_folds=5,shuffle=True, )
clf = MultinomialNB()
# clf = BernoulliNB()
# clf = GaussianNB()

In [65]:
for i, (train_index, test_index) in enumerate(kf):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    clf.fit(X_train, y_train.ravel())
    
    
    from sklearn.metrics import accuracy_score
    print(accuracy_score(clf.predict(X_test), y_test))

TRAIN: 2738 TEST: 685
0.810218978102
TRAIN: 2738 TEST: 685
0.792700729927
TRAIN: 2738 TEST: 685
0.778102189781
TRAIN: 2739 TEST: 684
0.763157894737
TRAIN: 2739 TEST: 684
0.788011695906
